In [ ]:
"""Bi-temporal Example

Demonstration of how the asAt date can be used to get data from different system dates.

Attributes
----------
transactions
holdings
bi-temporality
cocoon - identify_cash_items
transaction configuration
"""

# As at Date demonstration Example

This notebook will demonstrate how the As At date can be used to get data from different system dates.

An asset manager has a portfolio into which they load daily transactions for a week, including a corporate action dividend. Towards the end of the week, two corrections were made to the transaction posted on Day 1. At the end of the week they want to generate holdings for each day, calculated before and after each correction was made.

## Load transactions
- Day 1 :
Load transactions
- Day 2 :
Load Dividend corporate action and Funds In
- Day 3 :
nothing logged
- Day 4 :
Load in more transactions, including a transaction that occured on Day 3
- Day 5 : 
Correct transaction for Day 1

### Get holdings of Day 1 calculated each day of the week
The asset manager can track how the holdings on Day 1 changed throughout the week as corrections were made by calling the `get holdings` endpoint with effective date `Day 1` and As At dates on the following days

|   | As-At: 1  |  As-At: 2 |  As-At: 3 | As-At: 4  | As-At: 5  |
|---|---|---|---|---|---|
| Eff: 1  |  hld | hld | hld | hld | hld |

### Get holdings for each day, calculated any day of the week
The same approach can be used to get analyse corrections made for every other day of the week.

|   | As-At: 1  |  As-At: 2 |  As-At: 3 | As-At: 4  | As-At: 5  |
|---|---|---|---|---|---|
| Eff: 1  |  hld | hld | hld | hld | hld |
|  Eff: 2 |  - |  hld | hld | hld | hld |
|  Eff: 3 |  - |  - |  hld | hld | hld |
|  Eff: 4 | -  | -  | -  | hld  | hld |
|  Eff: 5 | -  | -  | -  |  - | hld  |



Firstly we begin by importing the modules we will be using. This tutorial will use `lusidtools`, a package that contains utility functions for preparing and uploading data using the lusid-python-sdk.

In [1]:
import os
import lusid
import copy
import matplotlib.pyplot as plt
from datetime import timedelta
from lusidtools.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import create_scope_id, identify_cash_items
from lusid.utilities import ApiClientFactory
from lusidtools.cocoon.dateorcutlabel import DateOrCutLabel
from lusidjam import RefreshingToken
import pandas as pd
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response
)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Define our scope and portfolio code
scope = "As_at_DEMO-"+ create_scope_id()
code = "PORT_0001"
print(f"Scope: {scope}")
print(f"Code: {code}")

LUSID Environment Initialised
LUSID version :  0.5.4243.0
Scope: As_at_DEMO-3857-f28e-d59f-e4
Code: PORT_0001


## Load Transaction data for each day

In [2]:
convert = {
    'txn_date': str,
    'settle_date': str,
}

# Load Data for each day
txn_data_1 = pd.read_excel("./data/as-at-POC-txns.xlsx", sheet_name="Day1", converters=convert)
txn_data_2 = pd.read_excel("./data/as-at-POC-txns.xlsx", sheet_name="Day2", converters=convert)
txn_data_3 = pd.read_excel("./data/as-at-POC-txns.xlsx", sheet_name="Day3", converters=convert)
txn_data_4 = pd.read_excel("./data/as-at-POC-txns.xlsx", sheet_name="Day4", converters=convert)
txn_data_5 = pd.read_excel("./data/as-at-POC-txns.xlsx", sheet_name="Day5", converters=convert)


This dictionary maps the required LUSID fields to the columns in the data files

In [3]:
mapping = {
    "instruments" : {
        "identifier_mapping": {
            "ClientInternal": "Internal_id",
        },
        "required": {
            "name": "instr_id"
        },
    },
    "portfolios": {
        "required": {
            "code": "Portfolio",
            "display_name":  "$As-at_POC",
            "base_currency": "$GBP",
            "created": "$2018-01-01T00:00:00+00:00"
        },
    },
    "transactions": {
        "identifier_mapping" : {
            "ClientInternal" : "Internal_id",
            "Name" : "instr_id"
        },
        "required" : {
            "code": "Portfolio",
            "transaction_id": "txn_id",
            "type": "type",
            "transaction_price.price": "price",
            "transaction_price.type": "$Price",                   # example of literal values
            "total_consideration.amount": "amount",
            "units": "units",
            "transaction_date": "txn_date",
            "transaction_currency": "trade_currency",
            "total_consideration.currency": "trade_currency",
            "settlement_date": "settle_date"
        },
        "properties": [
            "description"
        ]
    },
    "cash_flag": {
        "cash_identifiers": {
            "Internal_id" : ["GBP"]
        },
        "implicit": "trade_currency"
    }
}

# Load Instrument master

We will begin by uploading our instruments to LUSID from our security master. Once uploaded, LUSID will attempt to resolve any transactions or holdings against this. Any new instruments that cannot be resolved will be given a sudo ID until the updated security master is loaded, at which point any unresolved instruments will be updated. 

See the following support articles for more details on resolving instruments in LUSID:

[Resolving instruments in LUSID](https://support.finbourne.com/how-does-lusid-resolve-instruments)

[Unknown  instruments in LUSID](https://support.finbourne.com/what-is-the-unknown-instrument)

We have enough detail in the transaction file to populate a security master, so the initial instruments will be upserted using the transaction file as the source data.


In [4]:
# Data to be used to populate instrument master
txn_data_4.drop_duplicates(subset=[mapping["instruments"]["identifier_mapping"]["ClientInternal"]])

,Portfolio,txn_id,txn_date,settle_date,units,price,amount,trade_currency,instr_id,Internal_id,type,description
0,PORT_0001,txn_0002,2020-03-04,2020-03-05,100,20,2000,GBP,AVIVA,ID_aa001,Buy,NaN
1,PORT_0001,txn_0003,2020-03-03,2020-03-05,100,50,2000,GBP,VOD,ID_aa002,Buy,NaN
2,PORT_0001,txn_0004,2020-03-04,2020-03-05,100,80,2000,GBP,BP,ID_aa003,Buy,NaN


In [5]:
# Call LUSID to upsert instruments
response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=copy.deepcopy(txn_data_4.drop_duplicates(subset=[mapping["instruments"]["identifier_mapping"]["ClientInternal"]])),
        mapping_required=mapping["instruments"]["required"],
        file_type="instruments",
        mapping_optional={},
        identifier_mapping=mapping["instruments"]["identifier_mapping"]
)

# format response object
success, failed, errors = format_instruments_response(response)

pd.DataFrame(data=[{"success": len(success), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,3,0,0


## Create Portfolio

Next we will create a portfolio in LUSID into which we will upsert transactions.

In [6]:
# call LUSID to create a portfolio or update details of an existing portfolio
response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=copy.deepcopy(txn_data_1),
        mapping_required=mapping["portfolios"]["required"],
        file_type="portfolios",
        mapping_optional={},
)

# format response object
success, errors = format_portfolios_response(response)

pd.DataFrame(data=[{"success": len(success), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


# Upsert transactions

With a portfolio created in LUSID, We can now begin to upsert transactions on a daily basis.

## Day 1

Load transactions

Today one trade was made, 100 units of AVIVA. The details are upserted to LUSID using the following code.

In [7]:
# Day 1 transactions
txn_data_1

,Portfolio,txn_id,txn_date,settle_date,units,price,amount,trade_currency,instr_id,Internal_id,type,description
0,PORT_0001,txn_0001,2020-03-01,2020-03-01,100,20,2000,GBP,AVIVA,ID_aa001,StockIn,NaN


In [8]:
# call LUSID to upsert transactions
response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=txn_data_1,
        mapping_required=mapping["transactions"]["required"],
        file_type="transactions",
        mapping_optional={},
        identifier_mapping=mapping["transactions"]["identifier_mapping"]
)

# format response object
success, errors = format_transactions_response(response)

# Store the date at which these transactions were set
Day1_as_at_date = response["transactions"]["success"][0].version.as_at_date
    
pd.DataFrame(data=[{"success": len(success), "errors": len(errors)}])

,success,errors
0,1,0


## Day 2
Add funds and upload Dividend as a transaction

Today Funds were put into the account and a dividend was issued for AVIVA which in this case is represented as a transaction.

In [9]:
# Day 2 transactions
txn_data_2

,Portfolio,txn_id,txn_date,settle_date,units,price,amount,trade_currency,instr_id,Internal_id,type,description
0,PORT_0001,DIV_0001,2020-03-02,2020-03-02,200,1,200,GBP,GBP,ID_aa001,DividendCash,Dividend for AVIVA shareholders
1,PORT_0001,CASHIN_001,2020-03-02,2020-03-02,10000,1,10000,GBP,GBP,GBP,FundsIn,Adding Cash to portfolio


In [10]:
cash_processed_transactions, cash_processed_mapping = identify_cash_items(copy.deepcopy(txn_data_2), copy.deepcopy(mapping), "transactions", False)

response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=cash_processed_transactions,
        mapping_required=cash_processed_mapping["transactions"]["required"],
        file_type="transactions",
        mapping_optional={},
        identifier_mapping=cash_processed_mapping["transactions"]["identifier_mapping"]
)

success, errors = format_transactions_response(response)

# Store the date at which these transactions were set
Day2_as_at_date = response["transactions"]["success"][0].version.as_at_date
    
pd.DataFrame(data=[{"success": len(success), "errors": len(errors)}])

,success,errors
0,1,0


## Day 3

No trades or adjustments were recorded this day

In [11]:
# Day 3 transactions
txn_data_3

,Portfolio,txn_id,txn_date,settle_date,units,price,amount,trade_currency,instr_id,Internal_id,type


## Day 4
Load in more transactions including a Trade that occured on Day 3

In [12]:
# Day 4 transactions
txn_data_4

,Portfolio,txn_id,txn_date,settle_date,units,price,amount,trade_currency,instr_id,Internal_id,type,description
0,PORT_0001,txn_0002,2020-03-04,2020-03-05,100,20,2000,GBP,AVIVA,ID_aa001,Buy,NaN
1,PORT_0001,txn_0003,2020-03-03,2020-03-05,100,50,2000,GBP,VOD,ID_aa002,Buy,NaN
2,PORT_0001,txn_0004,2020-03-04,2020-03-05,100,80,2000,GBP,BP,ID_aa003,Buy,NaN


In [13]:
response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=txn_data_4,
        mapping_required=mapping["transactions"]["required"],
        file_type="transactions",
        mapping_optional={},
        identifier_mapping=mapping["transactions"]["identifier_mapping"]
)

success, errors = format_transactions_response(response)

# Store the date at which these transactions were set
Day4_as_at_date = response["transactions"]["success"][0].version.as_at_date

pd.DataFrame(data=[{"success": len(success), "errors": len(errors)}])

,success,errors
0,1,0


## Day 5
Make correction to `txn_0001` (units: 80 -> 120, price: 20 -> 25, amount: 2000 -> 3000)

In [14]:
# Day 5 transactions
txn_data_5

,Portfolio,txn_id,txn_date,settle_date,units,price,amount,trade_currency,instr_id,Internal_id,type,description
0,PORT_0001,txn_0001,2020-03-01,2020-03-01,120,25,3000,GBP,AVIVA,ID_aa001,StockIn,NaN


In [15]:
response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=txn_data_5,
        mapping_required=mapping["transactions"]["required"],
        file_type="transactions",
        mapping_optional={},
        identifier_mapping=mapping["transactions"]["identifier_mapping"]
)

success, errors = format_transactions_response(response)

# Store the date at which these transactions were set
Day5_as_at_date = response["transactions"]["success"][0].version.as_at_date

pd.DataFrame(data=[{"success": len(success), "errors": len(errors)}])

,success,errors
0,1,0


# Get holdings

While `StockIn` is a default transaction type for LUSID, `DividendCash`is a custom transaction type and we need to configure before getting holdings, in order for LUSID to understand how to interpret the underlying movements. See the support article on [configuring transaction types](https://support.finbourne.com/configuring-transaction-types) for more datail on this.

In [16]:
# Create a custom transaction type for cash dividends
txn_config_response =api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type(
    source="default",
    type="DividendCash",
    transaction_type_request=lusid.models.TransactionTypeRequest(
        aliases=[
            lusid.models.TransactionTypeAlias(
                type="DividendCash",
                description="A cash payments from dividends",
                transaction_class="EquityInstruments",
                transaction_roles="LongLonger",
            )
        ],
        movements=[
            lusid.models.TransactionTypeMovement(
                movement_types="CashCommitment",
                side="Side2",
                direction=1,
                properties=None,
                mappings=None,
            )
        ],
    )
)

In [17]:
# Define some helper functions 

from IPython.display import display_html

# Gets holdings for an effective date, calculated As At a list of days 
def get_holdings(api_factory, asat_dates, scope, code, effective_at):
    daily_report=[]
    n=1
    for i in asat_dates:
        data={}
        response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
            scope=scope,
            code=code,
            effective_at=effective_at,
            as_at=i.isoformat(),
            property_keys=["Instrument/default/Name"]
        )

        summary = display_holdings_summary(response)

        n=n+1
        daily_report.append(summary)
    return daily_report

# Prints the name and a quick summary from a get_holdings() response 
def display_holdings_summary(response):
    # inspect holdings response for today
    hld = [i for i in response.values]
    
    names=[]
    amount=[]
    units=[]
    price=[]
    holding_type=[]
    
    for item in hld:
        
        names.append(item.properties['Instrument/default/Name'].value.label_value)
        amount.append(item.cost.amount)
        units.append(item.units)
        holding_type.append(item.holding_type)
        
    data={
        "names" : names,
        "amount" : amount,
        "units" : units,
        "holding_type" : holding_type
    }
    
    summary = pd.DataFrame(data=data)
    return summary

# Displays Multiple dataframes side-by-side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table'),raw=True)

## Recovering Daily Positions

Now that all of the transactions (including corrections) have been loading into LUSID, the asset manager wants to **view the daily holdings as they apeared on that day**. 

### Day 1

On Day 1, 100 units of `AVIVA` stock was put into the portfolio

In [18]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
            scope=scope,
            code=code,
            effective_at="2020-03-01T00:00:00.000000+00:00",
            as_at=Day1_as_at_date,
            property_keys=["Instrument/default/Name"]
        )
display_side_by_side(display_holdings_summary(response))

,names,amount,units,holding_type
0,AVIVA,2000.0,100.0,P


### Day 2

On Day 2, A dividend was paid into the account of 200 GBP. In addition 10000 of GBP Funds were added to the account. This is represented as the currency `CCY_GBP` and has type `Cash Balance` (B)

See the support artical on [generating holdings](https://support.finbourne.com/how-are-holdings-generated-by-lusid) for more details on holding types.

In [19]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
            scope=scope,
            code=code,
            effective_at="2020-03-02T00:00:00.000000+00:00",
            as_at=Day2_as_at_date,
            property_keys=["Instrument/default/Name"]
        )
display_side_by_side(display_holdings_summary(response))

,names,amount,units,holding_type
0,AVIVA,2000.0,100.0,P
1,CCY_GBP,10200.0,10200.0,B


### Day 3
On day 3 no trades were logged

In [20]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
            scope=scope,
            code=code,
            effective_at="2020-03-03T00:00:00.000000+00:00",
            as_at=Day4_as_at_date - timedelta(microseconds=1),
            property_keys=["Instrument/default/Name"]
        )
display_side_by_side(display_holdings_summary(response))

,names,amount,units,holding_type
0,AVIVA,2000.0,100.0,P
1,CCY_GBP,10200.0,10200.0,B


### Day 4

Day 4's transactions file include:
- A backdated transaction from Day 3 in `VOD`: 100 units
- 2 transactions for Day 4:
    - `AVIVA`: 100 units 
    - `BP`: 100 units
    
This brings the total number of units in `AVIVA` to 180 (80 + 100)

The Buy transactions on Days 3 & 4 do not settle on the day on which the trades were placed. The cash used to buy these is represented as `Cash Commitments` (C) which are get processed on the day at which they settle.

In [21]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
            scope=scope,
            code=code,
            effective_at="2020-03-04T00:00:00.000000+00:00",
            as_at=Day4_as_at_date,
            property_keys=["Instrument/default/Name"]
        )
display_side_by_side(display_holdings_summary(response))

,names,amount,units,holding_type
0,AVIVA,4000.0,200.0,P
1,VOD,2000.0,100.0,P
2,BP,2000.0,100.0,P
3,CCY_GBP,10200.0,10200.0,B
4,CCY_GBP,-2000.0,-2000.0,C
5,CCY_GBP,-2000.0,-2000.0,C
6,CCY_GBP,-2000.0,-2000.0,C


### Day 5
Day 5 contained a correction to transaction `txn_0001` from Day 1 in `AVIVA`: 120 units

This brings the total number of units in `AVIVA` to 220 (120 + 100)

The Trades from Days 3 & 4 have also settled by Day 5 and the Cash Balance is adjusted acordingly

In [22]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
            scope=scope,
            code=code,
            effective_at="2020-03-05T00:00:00.000000+00:00",
            as_at=Day5_as_at_date,
            property_keys=["Instrument/default/Name"]
        )
display_side_by_side(display_holdings_summary(response))

,names,amount,units,holding_type
0,AVIVA,5000.0,220.0,P
1,VOD,2000.0,100.0,P
2,BP,2000.0,100.0,P
3,CCY_GBP,4200.0,4200.0,B


## Viewing the ammendments made to Day 1

The transaction on Day 1 was corected on Day 5. In order to see how our holdings for monday changed with these amendments, we will get holdings `effective at` Day 1 with the `as at` date specified as Day 1, Day 5.


In [25]:
# get holdings for Day 1 as at Day 1
response_day1_asat_day1 = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
                scope=scope,
                code=code,
                effective_at="2020-03-01T00:00:00.000000+00:00",
                as_at=Day1_as_at_date.isoformat(),
                property_keys=["Instrument/default/Name"]
            )

# get holdings for Day 1 as at Day 5
response_day1_asat_day5 = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
                scope=scope,
                code=code,
                effective_at="2020-03-01T00:00:00.000000+00:00",
                as_at=Day5_as_at_date.isoformat(),
                property_keys=["Instrument/default/Name"]
            )

print(f"Day 1, Calculated as at Day 1:")
display_side_by_side(display_holdings_summary(response_day1_asat_day1))

print(f"Day1, Calculated as at Day 5 (including correction):")
display_side_by_side(display_holdings_summary(response_day1_asat_day5))

Day 1, Calculated as at Day 1:


,names,amount,units,holding_type
0,AVIVA,2000.0,100.0,P


Day1, Calculated as at Day 5 (including correction):


,names,amount,units,holding_type
0,AVIVA,3000.0,120.0,P
